# GLM Demo: univariate stimulus, biophysical dynamics

Here we simulate data using the biophysical dynamical cascade model, and then try to estimate parameters with the GLM. The GLM does not have a membrane, so the estimated RF should be the convolution of the input kernel ($k1$) with the membrane kernel ($k2$) which is approximately an exponential decay with time constant $\tau_m$. However, for biophysical models with a low-threshold potassium current ($K_{LT}$), the kernel gets shifted to shorter lags for the phasic model, which is consistent with a variety of published studies showing that $K_{LT}$ acts as a bandpass filter.

This notebook also introduces the use of a raised cosine basis set to reduce the number of parameters in the kernel estimate.

In [ ]:
from __future__ import print_function, division
import sys
import imp
import os
import numpy as np
import scipy as sp
import scipy.signal as sps
import quickspikes as qs

import mat_neuron._model as mat
from dstrf import io, strf, mle, simulate, data, filters, models, spikes, performance
import spyks.core as spkc

# plotting packages
%reload_ext yamlmagic
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

cfg = {}

In [ ]:
%%yaml cfg
model:
  dt: 0.5
  ataus: [10.0, 200.0]
  t_refract: 2.0
  filter:
    len: 60
    ncos: 8
data:
  source: "randn"
  stimulus:    
    duration: 100000
    intro: 100
    random_seed: 1
  filter:
    fn: "gammadiff"
    tau1: 32.0
    tau2: 16.0
    amplitude: 10.0
    ntau: 60
    dt: 10.0
  model: "multivariate_dynamical"
  dynamics:
    model: "../../models/phasic.yml"
    current_scaling: 8.0
    current_compression:
      intercept: -1.32215976
      slope: 0.04068182
      V_lower: -100
      V_upper: 20
    current_recenter: 0.0    
  trial_noise:
    random_seed: 100
    snr: 2.0
  dt: 10.0
  trials: 10
spike_detect:
  thresh: -20.0
  rise_dt: 1.0

In [ ]:
from munch import munchify
cf = munchify(cfg)

In [ ]:
k1, k1t = filters.gammadiff(**cf.data.filter)
plt.plot(k1t, k1.T)

## Generate the stimulus and response

We are also using gaussian white noise here.

In [ ]:
import imp
imp.reload(io)
assim_data = data.randn(cf)
assim_data = io.merge_data(simulate.multivariate_dynamical(cf, assim_data))

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(9, 4))
t_stim = np.arange(0, cf.data.stimulus.duration, cf.data.dt)
t_voltage = np.linspace(0, cf.data.stimulus.duration, assim_data["V"].size)

axes[0].plot(t_stim, assim_data["stim"].T)
axes[1].plot(t_stim, assim_data["I"], t_voltage, assim_data["V"])
for i, spk in enumerate(assim_data["spike_t"]):
    axes[2].vlines(spk * cf.model.dt, i, i + 0.5)

axes[0].set_xlim(0, 8000);
print("spike count: {}".format(assim_data["spike_v"].sum()))

## Estimation

The theano code for setting up maximum likelihood estimation has been factored out into the `mle` package.

In [ ]:
# initial guess of parameters using regularized ML
kcosbas = strf.cosbasis(cf.model.filter.len, cf.model.filter.ncos)
try:
    mlest = mle.mat(assim_data["stim"], kcosbas, assim_data["spike_v"], assim_data["spike_h"],
                    assim_data["stim_dt"], assim_data["spike_dt"])
except TypeError:
    mlest = mle.mat(assim_data["stim"], kcosbas, assim_data["spike_v"], assim_data["spike_h"],
                    assim_data["stim_dt"], assim_data["spike_dt"])
# constrain to allowed region
#nparams = 1 + mlest.n_hparams + mlest.n_kparams
#constraint = models.matconstraint(nparams, cf.model.ataus[0], cf.model.ataus[1], cf.model.t_refract)
%time w0 = mlest.estimate(reg_alpha=1.0) #, method='trust-constr', constraints=[constraint])

In [ ]:
print("MLE rate and adaptation parameters:", w0[:3])
rf_ml = strf.from_basis(w0[3:], kcosbas)[::-1]
plt.plot(k1[0, ::-1], label="kernel")

# there is an expected shift due to the filtering properties of the membrane
stim_dt = cf.data.dt
model_dt = cf.model.dt
km, kmt = filters.exponential(46, 1.0, cf.data.filter.ntau * stim_dt, stim_dt)
kconv = np.convolve(km[::-1], k1[0, ::-1], mode="full")[:km.size]
kconv *= k1.max() / kconv.max()
plt.plot(kconv, label="expected")

plt.plot(rf_ml * k1.max() / rf_ml.max(), label="MLE")
plt.legend()

In [ ]:
test_stim = data.randn(cf, random_seed=1000)
test_data = io.merge_data(simulate.multivariate_dynamical(cf, test_stim, random_seed=1000, trials=10))
mltest = mle.mat(test_data["stim"], kcosbas, test_data["spike_v"], test_data["spike_h"],
                 test_data["stim_dt"], test_data["spike_dt"])

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True)
axes[0].plot(t_stim, test_data["stim"].T)

t_stim = np.linspace(0, test_data["duration"], test_data["stim"].shape[1])
t_spike = np.linspace(0, test_data["duration"], test_data["spike_v"].shape[0])

Vpred = mltest.V(w0)
n_trials = test_data["ntrials"]
for i, spk in enumerate(test_data["spike_t"]):
    axes[1].vlines(spk * cf.model.dt, i - 0.4 + n_trials, i + 0.4 + n_trials)
pred = np.zeros_like(test_data["spike_v"])
for j in range(n_trials):
    pred[:, j] = models.predict_spikes_glm(Vpred, w0[:3], cf)
    spk_t = pred[:, j].nonzero()[0]
    axes[1].vlines(spk_t * cf.model.dt, j - 0.4, j + 0.4, color='r')

upsample = int(cf.data.dt / cf.model.dt)   
pred_psth = spikes.psth(pred, upsample, 1)
test_psth = spikes.psth(test_data["spike_v"], upsample, 1)
axes[2].plot(t_stim, test_psth, t_stim, pred_psth)

axes[0].set_xlim(0, 4500);

eo = performance.corrcoef(test_data["spike_v"][::2], test_data["spike_v"][1::2], upsample, 1)
print("EO cc: %3.3f" % eo)
print("pred cc: %3.3f" % np.corrcoef(test_psth, pred_psth)[0, 1])
print("spike count: data = {}, pred = {}".format(test_data["spike_v"].sum(), pred.sum()))